# Working With PyTorch

Similarity to numpy, BboxTools can also compliable with pytorch. The method apply() and assign() that could be used on numpy array could also apply to pytorch tensor. Notice that the standard shape for numpy array is (h, w) or (h, w, c), however for torch tensor is (n, c, h, w) or (c, h, w) or (h, w). The bbox is also define on h and w axis.

In [1]:
import torch
import BboxTools as bbt

**Create bbox** \
Similar to getting start chapter, we will first create a bbox but apply it to torch tensor.

In [2]:
# Create a bbox with y: 2 to 5, x: 3 to 6, desire to working with image with size 8 * 8 
box = bbt.Bbox2D([(2, 5), (3, 6)], image_boundary=(8, 8))
print(box)

<class "Bbox2D", shape=[(2, 5), (3, 6)], boundary=[8, 8]>


In [3]:
a = torch.arange(0, 8).view(1, -1) + torch.arange(0, 8).view(-1, 1)
print(a)

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7],
        [ 1,  2,  3,  4,  5,  6,  7,  8],
        [ 2,  3,  4,  5,  6,  7,  8,  9],
        [ 3,  4,  5,  6,  7,  8,  9, 10],
        [ 4,  5,  6,  7,  8,  9, 10, 11],
        [ 5,  6,  7,  8,  9, 10, 11, 12],
        [ 6,  7,  8,  9, 10, 11, 12, 13],
        [ 7,  8,  9, 10, 11, 12, 13, 14]])


In [4]:
# Apply the bbox to tensor a
print(box.apply(a))

tensor([[5, 6, 7],
        [6, 7, 8],
        [7, 8, 9]])


In [5]:
# Increase value inside bbox by 20
box.assign(a, box.apply(a) + 20)
print(a)

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7],
        [ 1,  2,  3,  4,  5,  6,  7,  8],
        [ 2,  3,  4, 25, 26, 27,  8,  9],
        [ 3,  4,  5, 26, 27, 28,  9, 10],
        [ 4,  5,  6, 27, 28, 29, 10, 11],
        [ 5,  6,  7,  8,  9, 10, 11, 12],
        [ 6,  7,  8,  9, 10, 11, 12, 13],
        [ 7,  8,  9, 10, 11, 12, 13, 14]])


**Apply to 3D and 4D tensor** \
The common shape in CNN is (n, c, h, w). BboxTools can also be apply to this tensors while retain the gradient. Notice: a tensor requires grad cannot using assign method, as pytorch doesn't allow inplace operation on a tensor with enable_grad = True.

In [6]:
print(box.shape)

# Applying bbox to a 3D tensor
b = torch.zeros((6, 8, 8))
print(box.apply(b).shape)

# Applying bbox to a 4D tensor
b = torch.zeros((2, 6, 8, 8))
print(box.apply(b).shape)

(3, 3)
torch.Size([6, 3, 3])
torch.Size([2, 6, 3, 3])


In [7]:
# Assign bbox to a 4D tensor with value 1
box.assign(b, 1)

# The result should be 2 * 6 * 3 * 3 = 108
print(torch.sum(b))

tensor(108.)


In [8]:
# Applying bbox will retain grad
c = torch.ones((1, 1, 8, 8))
c.requires_grad = True
loss = torch.sum(box.apply(c))
loss.backward()
print(c.grad)

tensor([[[[0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 1., 1., 1., 0., 0.],
          [0., 0., 0., 1., 1., 1., 0., 0.],
          [0., 0., 0., 1., 1., 1., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.]]]])


**Create a bbox with torch tensor** \
You can also calculate a box that cover all the non-zero parts of a tensor.

In [9]:
d = c.grad
print(d)

# Create a box covers nonzero parts of d
print(bbt.nonzero(d))

tensor([[[[0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 1., 1., 1., 0., 0.],
          [0., 0., 0., 1., 1., 1., 0., 0.],
          [0., 0., 0., 1., 1., 1., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.]]]])
<class "Bbox2D", shape=[(2, 4), (3, 5)], boundary=[8, 8]>


In [10]:
# You can also use full method to a tensor
print(bbt.full(d))

<class "Bbox2D", shape=[(0, 8), (0, 8)], boundary=[8, 8]>
